In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm

In [2]:
label1 = pd.read_csv('../data/preliminary_train_label_dataset.csv')
label2 = pd.read_csv('../data/preliminary_train_label_dataset_s.csv')
label_df = pd.concat([label1, label2]).reset_index(drop=True)
label_df = label_df.drop_duplicates().reset_index(drop=True)

In [3]:
log_df = pd.read_csv('./log_template.csv')

In [4]:
submit_df = pd.read_csv('../data/preliminary_submit_dataset_a.csv')

In [5]:
log_df['time'] = pd.to_datetime(log_df['time'])
label_df['fault_time'] = pd.to_datetime(label_df['fault_time'])
submit_df['fault_time'] = pd.to_datetime(submit_df['fault_time'])

log_df['time_ts'] = log_df["time"].values.astype(np.int64) // 10 ** 9
label_df['fault_time_ts'] = label_df["fault_time"].values.astype(np.int64) // 10 ** 9
submit_df['fault_time_ts'] = submit_df["fault_time"].values.astype(np.int64) // 10 ** 9

In [6]:
log_df.shape, label_df.shape, submit_df.shape

((493527, 9), (16604, 4), (3011, 3))

In [7]:
log_df.sort_values(by=['sn', 'time'], inplace=True)

In [8]:
data = []
for i in tqdm(range(label_df.shape[0])):
    row = label_df.iloc[i]
    sn = row['sn']
    fault_time = row['fault_time']
    tmp = log_df[log_df['sn'] == sn]
    tmp = tmp[(tmp['time'] <= fault_time)].sort_values(by='time', ascending=False).tail(50)
    data.append(np.unique(tmp['template_id'].values.tolist()))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 16604/16604 [08:07<00:00, 34.09it/s]


In [9]:
label_df['template_ids'] = data

In [10]:
label_cnt_df = label_df.groupby('label').size().reset_index().rename({0:'label_cnt'},axis=1)

In [11]:
from collections import Counter

counters = []
for label, group in label_df.groupby('label'):
    token2count = Counter()
    for i in range(group.shape[0]):
        token2count.update(group.iloc[i]['template_ids'])
    counters.append(token2count)

In [12]:
tr_data = []
for i in tqdm(range(label_df.shape[0])):
    tr_data.append([])
    row = label_df.iloc[i]
    templates = row['template_ids']
    tr_data[-1] += [row['sn'], row['fault_time'], row['label']]
    for label in range(4):
        probas = [(counters[label][tid] / label_cnt_df.values[label][1]) for tid in templates]
        if len(probas) > 0:
            tr_data[-1] += [np.mean(probas), np.sum(probas), np.max(probas)]
        if len(probas) == 0:
            tr_data[-1] += [0, 0, 0]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 16604/16604 [00:07<00:00, 2359.53it/s]


In [13]:
tr_proba_df = pd.DataFrame(tr_data)

In [14]:
submit_df['fault_time'] = pd.to_datetime(submit_df['fault_time'])
data = []
for i in tqdm(range(submit_df.shape[0])):
    row = submit_df.iloc[i]
    sn = row['sn']
    fault_time = row['fault_time']
    tmp = log_df[log_df['sn'] == sn]
    tmp = tmp[(tmp['time'] <= fault_time)].sort_values(by='time', ascending=False).tail(50)
    data.append(np.unique(tmp['template_id'].values.tolist()))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 3011/3011 [01:30<00:00, 33.31it/s]


In [15]:
submit_df['template_ids'] = data

In [16]:
te_data = []
for i in tqdm(range(submit_df.shape[0])):
    te_data.append([])
    row = submit_df.iloc[i]
    te_data[-1] += [row['sn'], row['fault_time']]
    templates = row['template_ids']
    for label in range(4):
        probas = [(counters[label][tid] / label_cnt_df.values[label][1]) for tid in templates]
        if len(probas) > 0:
            te_data[-1] += [np.mean(probas), np.sum(probas), np.max(probas)]
        if len(probas) == 0:
            te_data[-1] += [0, 0, 0]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3011/3011 [00:01<00:00, 2666.12it/s]


In [17]:
te_proba_df = pd.DataFrame(te_data)

In [18]:
te_proba_df = pd.DataFrame(te_data)

In [19]:
columns = ['mean_proba_0', 'sum_proba_0', 'max_proba_0',
           'mean_proba_1', 'sum_proba_1', 'max_proba_1',
           'mean_proba_2', 'sum_proba_2', 'max_proba_2',
           'mean_proba_3', 'sum_proba_3', 'max_proba_3']

In [20]:
tr_proba_df.columns = ['sn', 'fault_time', 'label'] + columns

In [21]:
te_proba_df.columns = ['sn', 'fault_time'] + columns

In [22]:
tr_proba_df.to_csv('train4.csv', index=False)
te_proba_df.to_csv('test4.csv', index=False)

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm

In [2]:
label1 = pd.read_csv('../data/preliminary_train_label_dataset.csv')
label2 = pd.read_csv('../data/preliminary_train_label_dataset_s.csv')
label_df = pd.concat([label1, label2]).reset_index(drop=True)
label_df = label_df.drop_duplicates().reset_index(drop=True)

In [3]:
log_df = pd.read_csv('./new_log.csv')

In [4]:
submit_df = pd.read_csv('../data/preliminary_submit_dataset_a.csv')

In [5]:
log_df['time'] = pd.to_datetime(log_df['time'])
label_df['fault_time'] = pd.to_datetime(label_df['fault_time'])
submit_df['fault_time'] = pd.to_datetime(submit_df['fault_time'])

log_df['time_ts'] = log_df["time"].values.astype(np.int64) // 10 ** 9
label_df['fault_time_ts'] = label_df["fault_time"].values.astype(np.int64) // 10 ** 9
submit_df['fault_time_ts'] = submit_df["fault_time"].values.astype(np.int64) // 10 ** 9

In [6]:
log_df.shape, label_df.shape, submit_df.shape

((493527, 7), (16604, 4), (3011, 3))

In [7]:
log_df.sort_values(by=['sn', 'time'], inplace=True)

In [8]:
data = []
for i in tqdm(range(label_df.shape[0])):
    row = label_df.iloc[i]
    sn = row['sn']
    fault_time = row['fault_time']
    tmp = log_df[log_df['sn'] == sn]
    tmp = tmp[(tmp['time'] <= fault_time)].tail(10).sort_values(by='time', ascending=False)
    data.append(np.unique(tmp['msg_id'].values.tolist()))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16604/16604 [07:55<00:00, 34.91it/s]


In [9]:
label_df['msg_ids'] = data

In [10]:
label_cnt_df = label_df.groupby('label').size().reset_index().rename({0:'label_cnt'},axis=1)

In [11]:
from collections import Counter

counters = []
for label, group in label_df.groupby('label'):
    token2count = Counter()
    for i in range(group.shape[0]):
        token2count.update(group.iloc[i]['msg_ids'])
    counters.append(token2count)

In [12]:
tr_data = []
for i in tqdm(range(label_df.shape[0])):
    tr_data.append([])
    row = label_df.iloc[i]
    templates = row['msg_ids']
    tr_data[-1] += [row['sn'], row['fault_time'], row['label']]
    for label in range(4):
        probas = [(counters[label][tid] / label_cnt_df.values[label][1]) for tid in templates]
        if len(probas) > 0:
            tr_data[-1] += [np.mean(probas), np.sum(probas), np.max(probas)]
        if len(probas) == 0:
            tr_data[-1] += [0, 0, 0]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16604/16604 [00:06<00:00, 2717.20it/s]


In [13]:
tr_proba_df = pd.DataFrame(tr_data)

In [14]:
submit_df['fault_time'] = pd.to_datetime(submit_df['fault_time'])
data = []
for i in tqdm(range(submit_df.shape[0])):
    row = submit_df.iloc[i]
    sn = row['sn']
    fault_time = row['fault_time']
    tmp = log_df[log_df['sn'] == sn]
    tmp = tmp[(tmp['time'] <= fault_time)].tail(10).sort_values(by='time', ascending=False)
    data.append(np.unique(tmp['msg_id'].values.tolist()))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3011/3011 [01:28<00:00, 33.84it/s]


In [15]:
submit_df['msg_ids'] = data

In [16]:
te_data = []
for i in tqdm(range(submit_df.shape[0])):
    te_data.append([])
    row = submit_df.iloc[i]
    te_data[-1] += [row['sn'], row['fault_time']]
    templates = row['msg_ids']
    for label in range(4):
        probas = [(counters[label][tid] / label_cnt_df.values[label][1]) for tid in templates]
        if len(probas) > 0:
            te_data[-1] += [np.mean(probas), np.sum(probas), np.max(probas)]
        if len(probas) == 0:
            te_data[-1] += [0, 0, 0]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3011/3011 [00:01<00:00, 2962.28it/s]


In [17]:
te_proba_df = pd.DataFrame(te_data)

In [18]:
columns = ['mean_proba_0', 'sum_proba_0', 'max_proba_0',
           'mean_proba_1', 'sum_proba_1', 'max_proba_1',
           'mean_proba_2', 'sum_proba_2', 'max_proba_2',
           'mean_proba_3', 'sum_proba_3', 'max_proba_3']

In [19]:
tr_proba_df.columns = ['sn', 'fault_time', 'label'] + columns

In [20]:
te_proba_df.columns = ['sn', 'fault_time'] + columns

In [25]:
tr_proba_df.to_csv('train4.csv', index=False)
te_proba_df.to_csv('test4.csv', index=False)

In [21]:
tr_proba_df

,sn,fault_time,label,mean_proba_0,sum_proba_0,max_proba_0,mean_proba_1,sum_proba_1,max_proba_1,mean_proba_2,sum_proba_2,max_proba_2,mean_proba_3,sum_proba_3,max_proba_3
0,SERVER_25698,2020-10-09 13:43:00,0,0.000678,0.000678,0.000678,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,SERVER_25699,2020-08-25 18:50:00,0,0.002258,0.006775,0.003388,0.000197,0.00059,0.000295,0.000680,0.002039,0.001073,0.000688,0.002065,0.001652
2,SERVER_25712,2020-03-16 13:20:00,0,0.002439,0.012195,0.003388,0.000118,0.00059,0.000295,0.000773,0.003863,0.001502,0.001652,0.008261,0.003304
3,SERVER_25708,2020-07-25 12:44:00,0,0.002710,0.005420,0.003388,0.000000,0.00000,0.000000,0.000912,0.001824,0.001502,0.003098,0.006196,0.003304
4,SERVER_25711,2020-03-16 16:51:00,0,0.002258,0.006775,0.003388,0.000197,0.00059,0.000295,0.000715,0.002146,0.001073,0.000688,0.002065,0.001652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16599,SERVER_24971,2020-03-04 21:09:00,3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.003924,0.007848,0.004131
16600,SERVER_24971,2020-11-12 20:49:00,3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.003924,0.007848,0.004131
16601,SERVER_24962,2020-09-12 12:18:00,3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000826,0.001652,0.000826
16602,SERVER_24971,2020-10-04 17:41:00,3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.003924,0.007848,0.004131


In [22]:
te_proba_df

,sn,fault_time,mean_proba_0,sum_proba_0,max_proba_0,mean_proba_1,sum_proba_1,max_proba_1,mean_proba_2,sum_proba_2,max_proba_2,mean_proba_3,sum_proba_3,max_proba_3
0,000d33b21436,2020-09-02 16:42:54,0.251694,0.503388,0.268293,0.267198,0.534396,0.337762,0.108369,0.216738,0.115343,0.145808,0.291615,0.176373
1,005c5a9218ba,2020-06-28 19:05:16,0.078930,0.315718,0.235095,0.052332,0.209330,0.196634,0.033101,0.132403,0.101395,0.030256,0.121024,0.115242
2,0079283bde6e,2020-04-26 21:32:44,0.000678,0.000678,0.000678,0.001181,0.001181,0.001181,0.000107,0.000107,0.000107,0.104502,0.104502,0.104502
3,007bdf23b62f,2020-06-16 18:40:39,0.146206,0.731030,0.310298,0.157603,0.788013,0.371715,0.095944,0.479721,0.281974,0.106237,0.531185,0.243701
4,00a577a8e54f,2020-04-07 07:16:55,0.000790,0.004743,0.001355,0.006840,0.041039,0.011810,0.050733,0.304399,0.072639,0.037037,0.222222,0.068980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,ffbf46b4af21,2019-12-28 20:10:01,0.002033,0.002033,0.002033,0.004724,0.004724,0.004724,0.014485,0.014485,0.014485,0.000413,0.000413,0.000413
3007,ffc229b6cd9a,2020-06-27 02:39:08,0.054878,0.054878,0.054878,0.036315,0.036315,0.036315,0.281974,0.281974,0.281974,0.011152,0.011152,0.011152
3008,ffd44698a52b,2020-01-21 15:46:56,0.176152,0.352304,0.301491,0.189253,0.378506,0.364334,0.051073,0.102146,0.075751,0.135068,0.270136,0.212722
3009,fff73a9e5bd5,2020-03-01 22:43:43,0.054878,0.054878,0.054878,0.036315,0.036315,0.036315,0.281974,0.281974,0.281974,0.011152,0.011152,0.011152


In [8]:
data = []
for idx, row in label_df.iterrows():
    sn = row['sn']
    fault_time = row['fault_time']
    ts = row['fault_time_ts']
    df = log_df[log_df['sn'] == sn].copy()
    df = df[df['time_ts'] <= ts].copy()
    df = df.sort_values(by='time_ts').reset_index(drop=True).tail(5)
    text = '\n'.join(df['msg_lower'].values)
    data.append(text)

In [9]:
label_df['text'] = data

In [10]:
data = []
for idx, row in submit_df.iterrows():
    sn = row['sn']
    fault_time = row['fault_time']
    ts = row['fault_time_ts']
    df = log_df[log_df['sn'] == sn].copy()
    df = df[df['time_ts'] <= ts].copy()
    df = df.sort_values(by='time_ts').reset_index(drop=True).tail(5)
    text = '\n'.join(df['msg_lower'].values)
    data.append(text)

In [11]:
submit_df['text'] = data

In [12]:
label_cnt_df = label_df.groupby('label').size().reset_index().rename({0: 'label_cnt'}, axis=1)

In [13]:
label_df['text_list'] = label_df['text'].str.split()
submit_df['text_list'] = submit_df['text'].str.split()
label_df.shape, submit_df.shape

((16604, 6), (3011, 5))

In [14]:
exp_label_df = label_df.explode('text_list').reset_index(drop=True).drop_duplicates()
exp_submit_df = submit_df.explode('text_list').reset_index(drop=True).drop_duplicates()
exp_label_df.shape, exp_submit_df.shape

((225493, 6), (32474, 5))

In [15]:
exp_label_df.head()

,sn,fault_time,label,fault_time_ts,text,text_list
0,SERVER_25698,2020-10-09 13:43:00,0,1602250980,system boot initiated bios_boot_up | state ass...,system
1,SERVER_25698,2020-10-09 13:43:00,0,1602250980,system boot initiated bios_boot_up | state ass...,boot
2,SERVER_25698,2020-10-09 13:43:00,0,1602250980,system boot initiated bios_boot_up | state ass...,initiated
3,SERVER_25698,2020-10-09 13:43:00,0,1602250980,system boot initiated bios_boot_up | state ass...,bios_boot_up
4,SERVER_25698,2020-10-09 13:43:00,0,1602250980,system boot initiated bios_boot_up | state ass...,|


In [16]:
label_word_cnt_df = exp_label_df.groupby(['label', 'text_list']).size().reset_index()\
    .rename({0: 'label_word_cnt'}, axis=1)
label_word_cnt_df.shape

(1149, 3)

In [17]:
label_word_cnt_df = label_word_cnt_df.merge(label_cnt_df, how='left', on='label')
label_word_cnt_df.shape

(1149, 4)

In [18]:
label_word_cnt_df['p(label_word_cnt|label_cnt)'] = \
    label_word_cnt_df['label_word_cnt'] / label_word_cnt_df['label_cnt']
label_word_cnt_df.shape

(1149, 5)

In [19]:
label_word_cnt_df.head()

,label,text_list,label_word_cnt,label_cnt,p(label_word_cnt|label_cnt)
0,0,#0x04,1,1476,0.000678
1,0,#0x07,4,1476,0.002710
2,0,#0x10,54,1476,0.036585
3,0,#0x16,41,1476,0.027778
4,0,#0x17,71,1476,0.048103


In [24]:
feature_df = None
for i in range(4):
    exp_label_df['label'] = i
    tmp = exp_label_df.merge(label_word_cnt_df, how='left', on=['label', 'text_list']).fillna(0)
    tmp = tmp.groupby(['sn', 'fault_time', 'label'])['p(label_word_cnt|label_cnt)']
    tmp = tmp.agg([np.mean, np.sum, np.max]).reset_index()
    tmp = tmp[['sn', 'fault_time', 'mean', 'sum', 'amax']].rename({
        'mean': 'mean_%d'%i,
        'sum': 'sum_%d'%i,
        'amax': 'amax_%d'%i
    }, axis=1)
    if feature_df is None:
        feature_df = tmp
    else:
        feature_df = feature_df.merge(tmp, on=['sn', 'fault_time'])

In [25]:
label_df = label_df.merge(feature_df, on=['sn', 'fault_time'])

In [26]:
feature_df = None
for i in range(4):
    exp_submit_df['label'] = i
    tmp = exp_submit_df.merge(label_word_cnt_df, how='left', on=['label', 'text_list']).fillna(0)
    tmp = tmp.groupby(['sn', 'fault_time', 'label'])['p(label_word_cnt|label_cnt)']
    tmp = tmp.agg([np.mean, np.sum, np.max]).reset_index()
    tmp = tmp[['sn', 'fault_time', 'mean', 'sum', 'amax']].rename({
        'mean': 'mean_%d'%i,
        'sum': 'sum_%d'%i,
        'amax': 'amax_%d'%i
    }, axis=1)
    if feature_df is None:
        feature_df = tmp
    else:
        feature_df = feature_df.merge(tmp, on=['sn', 'fault_time'])

In [27]:
submit_df = submit_df.merge(feature_df, on=['sn', 'fault_time'])

In [28]:
label_df.shape

(16604, 18)

In [29]:
submit_df.shape

(3011, 17)

In [30]:
features = []
for i in range(4):
    features += ['mean_%d'%i, 'sum_%d'%i, 'amax_%d'%i,]

In [32]:
label_df = label_df[['sn', 'fault_time', 'label'] + features]
submit_df = submit_df[['sn', 'fault_time'] + features]

In [23]:
label_df.to_csv('train4.csv', index=False)
submit_df.to_csv('test4.csv', index=False)